## Day 8: Haunted Wasteland

You're still riding a camel across Desert Island when you spot a sandstorm quickly approaching. When you turn to warn the Elf, she disappears before your eyes! To be fair, she had just finished warning you about ghosts a few minutes ago.

One of the camel's pouches is labeled "maps" - sure enough, it's full of documents (your puzzle input) about how to navigate the desert. At least, you're pretty sure that's what they are; one of the documents contains a list of left/right instructions, and the rest of the documents seem to describe some kind of network of labeled nodes.

It seems like you're meant to use the left/right instructions to navigate the network. Perhaps if you have the camel follow the same instructions, you can escape the haunted wasteland!

After examining the maps for a bit, two nodes stick out: AAA and ZZZ. You feel like AAA is where you are now, and you have to follow the left/right instructions until you reach ZZZ.

This format defines each node of the network individually. For example:

    RL

    AAA = (BBB, CCC)
    BBB = (DDD, EEE)
    CCC = (ZZZ, GGG)
    DDD = (DDD, DDD)
    EEE = (EEE, EEE)
    GGG = (GGG, GGG)
    ZZZ = (ZZZ, ZZZ)
    
Starting with AAA, you need to look up the next element based on the next left/right instruction in your input. In this example, start with AAA and go right (R) by choosing the right element of AAA, CCC. Then, L means to choose the left element of CCC, ZZZ. By following the left/right instructions, you reach ZZZ in 2 steps.

Of course, you might not find ZZZ right away. If you run out of left/right instructions, repeat the whole sequence of instructions as necessary: RL really means RLRLRLRLRLRLRLRL... and so on. For example, here is a situation that takes 6 steps to reach ZZZ:

LLR

AAA = (BBB, BBB)
BBB = (AAA, ZZZ)
ZZZ = (ZZZ, ZZZ

Starting at AAA, follow the left/right instructions. How many steps are required to reach ZZZ?

### Load in the Data

In [1]:
# load in sample puzzle input
file_path = 'data/movements.txt' 

try:
    with open(file_path, 'r') as file:
        movements = file.read()
except FileNotFoundError:
    print(f"File not found: {file_path}")
    movements = None

In [2]:
# load in sample puzzle input
file_path = 'data/locations.txt' 

try:
    with open(file_path, 'r') as file:
        locations = file.read()
except FileNotFoundError:
    print(f"File not found: {file_path}")
    locations = None

In [3]:
print(movements)

LRRRLRRRLRRLRLRRLRLRRLRRLRLLRRRLRLRLRRRLRRRLRLRLRLLRRLLRRLRRRLLRLRRRLRLRLRRRLLRLRRLRRRLRLRRRLLRLRRLRRRLRRLRRLRLRRLRRRLRLRRRLRRLLRRLRRLRLRRRLRRLRRRLRRRLRLRRLRLRRRLRLRRLRRLRRRLRRRLRRRLLRRLRRRLRLRLRLRRRLRLRLRRLRRRLRRRLRRLRRLLRLRRLLRLRRLRRLLRLLRRRLLRRLLRRLRRLRLRLRRRLLRRLRRRR


### Convert the movements string into a list of indexes L=0, R=1 to aid iteration through locations

In [4]:
movements = "LRRRLRRRLRRLRLRRLRLRRLRRLRLLRRRLRLRLRRRLRRRLRLRLRLLRRLLRRLRRRLLRLRRRLRLRLRRRLLRLRRLRRRLRLRRRLLRLRRLRRRLRRLRRLRLRRLRRRLRLRRRLRRLLRRLRRLRLRRRLRRLRRRLRRRLRLRRLRLRRRLRLRRLRRLRRRLRRRLRRRLLRRLRRRLRLRLRLRRRLRLRLRRLRRRLRRRLRRLRRLLRLRRLLRLRRLRRLLRLLRRRLLRRLLRRLRRLRLRLRRRLLRRLRRRR"

# Convert 'L' to 0 and 'R' to 1 using a lambda function
movements_list = list(map(lambda x: 1 if x == 'L' else 2, movements))

print(movements_list)

[1, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 1, 2, 1, 2, 2, 1, 2, 1, 2, 2, 1, 2, 2, 1, 2, 1, 1, 2, 2, 2, 1, 2, 1, 2, 1, 2, 2, 2, 1, 2, 2, 2, 1, 2, 1, 2, 1, 2, 1, 1, 2, 2, 1, 1, 2, 2, 1, 2, 2, 2, 1, 1, 2, 1, 2, 2, 2, 1, 2, 1, 2, 1, 2, 2, 2, 1, 1, 2, 1, 2, 2, 1, 2, 2, 2, 1, 2, 1, 2, 2, 2, 1, 1, 2, 1, 2, 2, 1, 2, 2, 2, 1, 2, 2, 1, 2, 2, 1, 2, 1, 2, 2, 1, 2, 2, 2, 1, 2, 1, 2, 2, 2, 1, 2, 2, 1, 1, 2, 2, 1, 2, 2, 1, 2, 1, 2, 2, 2, 1, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 1, 2, 1, 2, 2, 1, 2, 1, 2, 2, 2, 1, 2, 1, 2, 2, 1, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 1, 1, 2, 2, 1, 2, 2, 2, 1, 2, 1, 2, 1, 2, 1, 2, 2, 2, 1, 2, 1, 2, 1, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 1, 2, 2, 1, 1, 2, 1, 2, 2, 1, 1, 2, 1, 2, 2, 1, 2, 2, 1, 1, 2, 1, 1, 2, 2, 2, 1, 1, 2, 2, 1, 1, 2, 2, 1, 2, 2, 1, 2, 1, 2, 1, 2, 2, 2, 1, 1, 2, 2, 1, 2, 2, 2, 2]


In [13]:
print(len(movements))

271


### Convert the locations string into a list of indexes to be ierated through based on movements list

In [5]:
# Split the input string by lines and remove empty lines
lines = [line.strip() for line in locations.split('\n') if line.strip()]

# Create a list to store the result
locations_list = []

# Iterate through each line and process the data
for line in lines:
    # Split the line into parts using '=' and remove extra spaces
    parts = [part.strip() for part in line.split('=')]
    
    # Split the second part (inside parentheses) into individual items
    items = [item.strip('() ') for item in parts[1].split(',')]
    
    # Add the items to the result list
    locations_list.append([parts[0]] + items)

print(locations_list)

[['DBQ', 'RTP', 'NBX'], ['NFX', 'PXX', 'PLG'], ['VBK', 'BRV', 'DKG'], ['BRS', 'HLR', 'VBX'], ['DDK', 'SPR', 'TCR'], ['FTS', 'LJB', 'MDJ'], ['BLH', 'DFM', 'GGG'], ['PCC', 'GQR', 'RHD'], ['QKN', 'VVR', 'GBL'], ['KHN', 'FNB', 'LLT'], ['HTD', 'NPJ', 'BTL'], ['FPL', 'BRX', 'XQD'], ['RCJ', 'QKN', 'XPD'], ['XRN', 'RMQ', 'LQB'], ['HGM', 'VPV', 'SVR'], ['RFG', 'TLM', 'KPH'], ['SSG', 'QGC', 'KJV'], ['LPA', 'QQN', 'GNF'], ['SVN', 'VLM', 'BDX'], ['VPF', 'TTR', 'RNQ'], ['VNK', 'DSX', 'MCZ'], ['KMS', 'XNN', 'MTB'], ['RLS', 'NHX', 'CRT'], ['VRS', 'VBX', 'HLR'], ['TVK', 'MKG', 'NKB'], ['CKH', 'DFF', 'XPV'], ['KBS', 'LJC', 'PRS'], ['JMT', 'FPT', 'DLX'], ['LRV', 'QCD', 'RLF'], ['TMM', 'VPF', 'GKD'], ['HKH', 'PRS', 'LJC'], ['GSM', 'SVH', 'KRP'], ['SLG', 'PTL', 'SNJ'], ['GVS', 'DSN', 'CKH'], ['QGC', 'HKS', 'DFP'], ['RFT', 'VVQ', 'GMF'], ['FDF', 'JKV', 'JKV'], ['MRL', 'PKS', 'TKS'], ['HBN', 'BDX', 'VLM'], ['CFB', 'PPV', 'QVS'], ['TFJ', 'KGN', 'JND'], ['HLB', 'PFV', 'JXR'], ['RFJ', 'SKL', 'QCN'], ['PQH', 'K

### Create functions required to track current location and movements 

In [19]:
def find_next_location(locations_list, current_location, movement):
    start_index = next((i for i, sublist in enumerate(locations_list) if sublist[0] == current_location), None)
    if start_index is not None:
        return locations_list[start_index][movement] # Chat GPT has -1
    else:
        print(f"Location '{current_location}' not found.")
        return None

def find_index_of_value(locations_list, value):
    return next((i for i, sublist in enumerate(locations_list) if value in sublist), None) 

    
def move_through_locations(locations_list, movements_list, start_location):
    current_location = start_location
    result_list = [current_location]

    for movement in movements_list:
        current_location = find_next_location(locations_list, current_location, movement)
        if current_location is None:
            break
        result_list.append(current_location)

    return result_list[1:]  # Skip the first location to remove the overlap

overall_index = 0
all_locations = []

while True:
    # Find the list item that has the starting location at position 0 or use the end location of the previous loop
    start_location = 'AAA' if overall_index == 0 else result[-1]
    start_index = find_index_of_value(locations_list, start_location)

    if start_index is not None:
        result = move_through_locations(locations_list, movements_list, start_location)
        all_locations.extend(result)

        # Check if 'ZZZ' appears in the result
        if 'ZZZ' in result:
            zzz_index = all_locations.index('ZZZ')
            print(f"Overall Index of 'ZZZ': {zzz_index}")
            break
        else:
            print("'ZZZ' does not appear in the result. Starting the loop again.")
            overall_index += len(result) - 1
    else:
        print(f"Starting position '{start_location}' not found.")
        break

print("All locations:", all_locations)

'ZZZ' does not appear in the result. Starting the loop again.
'ZZZ' does not appear in the result. Starting the loop again.
'ZZZ' does not appear in the result. Starting the loop again.
'ZZZ' does not appear in the result. Starting the loop again.
'ZZZ' does not appear in the result. Starting the loop again.
'ZZZ' does not appear in the result. Starting the loop again.
'ZZZ' does not appear in the result. Starting the loop again.
'ZZZ' does not appear in the result. Starting the loop again.
'ZZZ' does not appear in the result. Starting the loop again.
'ZZZ' does not appear in the result. Starting the loop again.
'ZZZ' does not appear in the result. Starting the loop again.
'ZZZ' does not appear in the result. Starting the loop again.
'ZZZ' does not appear in the result. Starting the loop again.
'ZZZ' does not appear in the result. Starting the loop again.
'ZZZ' does not appear in the result. Starting the loop again.
'ZZZ' does not appear in the result. Starting the loop again.
'ZZZ' do

### Calculate the number of steps taken to get from AAA to ZZZZ

In [20]:
print(len(all_locations))

12737
